# Random-graph UD-MIS (Classical)

$$
H = -\sum_{i \in V} n_i + u \sum_{i,j \in E} n_i n_j
$$

Our goal is to find one of the ground state(s) - a solution to the UD-MIS problem - automatically through Monte Carlo sampling.

The probability distribution of the Ising model at temperature $T$ is given by the Gibbs distribution:

$$p(\sigma; T) = \frac{1}{Z(T)}\exp\left(-\frac{H(\sigma)}{T}\right)$$,

where $Z(T)$ is the normalization constant (also known as the *partition function*).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from abstract_udmis import AbstractUDMIS

%matplotlib inline

Below you'll find code for the UDMIS model of interest. Strictly speaking, for the purposes of performing the Monte Carlo simulation we can simply compute the required energy differences using two calls to the energy function. However, we can often save significant computation time by writing a specialised function for the energy difference that avoids unnecessary calculations. Simply put, calculating the energy of a configuration of occupations requires you to iterate over all vertices in the graph, but the energy change associated with a single-occupation-flip can be computed just from values of the connected vertices.

In [2]:
class UDMIS(AbstractUDMIS):
    def __init__(self, u, graph):
        super().__init__()
        self.u, self.graph = u, graph
        self.num_vertices = len(self.graph)
        print(self.num_vertices)
        # initialize system at infinite temperature
        # i.e. vertices are completely random and uncorrelated
        self.occupations = np.random.rand(self.num_vertices) < 0.5
        self.edges = self.find_edges()
    
    def find_edges(self):
        #num_pairs = int(self.num_vertices*(self.num_vertices)*0.5)
        edges = np.zeros((self.num_vertices, self.num_vertices), dtype=bool)
        
        for i in range(self.num_vertices-1):
            x_i, y_i = graph[i] # these are the x, y coordinates of the i'th vertex in the graph
            for j in range(i+1, self.num_vertices):
                x_j, y_j = graph[j] # these are the x, y coordinates of the j'th vertex in the graph
                
                # calculate the distance between vertices
                dij = np.sqrt((x_i - x_j)**2. + (y_i - y_j)**2.)
                if dij <= 1.0:
                    edges[i,j] = True
                    edges[j,i] = True
                    
        return edges
        
    def energy(self):
        """Returns the energy of the current spin configuration"""
        # interaction term
        interaction_term = 0
        vertex_term = 0
        for i in range(self.num_vertices-1):
            for j in range(i+1, self.num_vertices):
                
                # check if there is an edge
                if self.edges[i,j]:
                    interaction_term += self.occupations[i]*self.occupations[j]
                
            vertex_term += self.occupations[i]
        
        # missed the last vertex
        vertex_term += self.occupations[self.num_vertices-1]
    
        return u*interaction_term - vertex_term

    def energy_diff(self, i):
        connections = np.where(self.edges[i,:])[0]
        num_adjacent_occupied = sum(self.occupations[connections])
        
        if self.occupations[i] == 1:
            # flipping an occupied vertex increases the vertex term, decreases the interaction term
            vertex_term_change = 1.
            interaction_term_change = -u*num_adjacent_occupied
        
        elif self.occupations[i] == 0:
            # flipping an unoccupied vertex decreases the vertex term, increases the interaction term
            vertex_term_change = -1.
            interaction_term_change = u*num_adjacent_occupied 

        return interaction_term_change + vertex_term_change
    
    def rand_vertex(self):
        """Selects a site in the graph at random"""
        return np.random.randint(self.num_vertices)        

In [3]:
u = 1.35
graph = [(0.3461717838632017, 1.4984640297338632), 
         (0.6316400411846113, 2.5754677320579895), 
         (1.3906262250927481, 2.164978861396621), 
         (0.66436005100802, 0.6717919819739032), 
         (0.8663329771713457, 3.3876341010035995), 
         (1.1643107343501296, 1.0823066243402013)
        ]

udmis = UDMIS(u, graph)

6


In [18]:
N = 50000
t = np.arange(N+1)
T_i = 1000
T_f = 0.01

T = T_i * ((T_f/T_i) ** (t/N))

for t in range(N):
    # take a look at the abstract_udmis.py file to see how mc_step works
    temp = T[t]
    E = udmis.mc_step(T=temp)
    
    if t % 100 == 1:
        print(t,temp, E, udmis.occupations)

1 999.7697679981566 0.40000000000000036 [ True  True False  True  True  True]
101 977.0122296741541 -1.65 [ True False False  True  True False]
201 954.7727161666106 -1.0 [False False False  True False False]
301 933.0394357910897 -1.0 [ True False False False False False]
401 911.800865274662 -1.0 [False False False False  True False]
501 891.0457436461143 -2.9999999999999996 [ True False  True False  True False]
601 870.7630662652324 -1.65 [False  True  True False False  True]
701 850.9420789879961 -0.6499999999999999 [ True False False False False  True]
801 831.5722724645892 -0.6499999999999999 [False  True  True False False False]
901 812.6433765672033 -1.2999999999999998 [ True  True  True  True False False]
1001 794.1453549446802 0.75 [ True  True  True  True  True  True]
1101 776.0683997011051 -1.2999999999999998 [ True  True False  True  True False]
1201 758.4029261955313 -1.0 [False  True False False False False]
1301 741.1395679600757 1.0500000000000007 [ True False False  T

12401 57.53074526484432 -1.65 [False False  True  True False  True]
12501 56.221185622133106 -0.6499999999999995 [ True False False  True False False]
12601 54.94143519621414 -0.6499999999999999 [ True False False  True False False]
12701 53.69081544647207 -1.0 [False  True False False False False]
12801 52.46866327776168 -2.0 [False  True False False False  True]
12901 51.27433068882602 -1.0 [False False  True False False False]
13001 50.10718442871761 -0.6499999999999999 [False  True  True False False False]
13101 48.96660566104027 -2.0 [ True  True False False False False]
13201 47.85198963583406 -1.65 [ True  True False  True False False]
13301 46.76274536892872 -3.0 [False False  True False  True  True]
13401 45.69829532859612 -1.65 [ True  True False False False  True]
13501 44.658075129335174 -1.65 [ True  True False  True False False]
13601 43.64153323262731 -1.65 [False  True False  True  True False]
13701 42.64813065450337 -0.9499999999999997 [ True  True  True  True  True Fa

23801 4.167734067976718 -2.0 [False False  True False False  True]
23901 4.072864858272424 -2.65 [ True False  True  True  True False]
24001 3.9801551354267253 -2.65 [ True False  True False  True  True]
24101 3.8895557435174113 -2.65 [ True False  True  True  True False]
24201 3.801018645547667 -1.2999999999999998 [ True  True False False  True  True]
24301 3.7144968979762214 -0.6499999999999999 [ True False False False False  True]
24401 3.629944625827262 -1.2999999999999998 [False  True  True False  True  True]
24501 3.5473169983669135 -1.65 [ True False  True  True False False]
24601 3.4665702053333907 -3.0 [ True False  True False  True False]
24701 3.387661433708216 -1.65 [ True False  True  True False False]
24801 3.3105488450161946 -1.2999999999999998 [ True  True False  True  True False]
24901 3.2351915531420947 -1.65 [ True False  True False False  True]
25001 3.1615496026522933 -1.65 [ True  True False False  True False]
25101 3.0895839476098748 -1.2999999999999998 [ True  T

36001 0.25113081148680505 -3.0 [False False  True  True  True False]
36101 0.24541437631374297 -3.0 [False False  True False  True  True]
36201 0.2398280630914457 -3.0 [False False  True  True  True False]
36301 0.23436890988269923 -2.65 [ True False  True  True  True False]
36401 0.22903402217221183 -1.65 [False  True False  True  True False]
36501 0.2238205713319038 -3.0 [ True False  True False  True False]
36601 0.21872579312113144 -2.65 [ True False  True  True  True False]
36701 0.21374698622105007 -3.0 [False False  True  True  True False]
36801 0.2088815108023389 -2.0 [False False False False  True  True]
36901 0.2041267871255288 -3.0 [False False  True  True  True False]
37001 0.19948029417319016 -3.0 [False False  True False  True  True]
37101 0.19493956831325593 -3.0 [ True False  True False  True False]
37201 0.19050220199277162 -2.65 [False False  True  True  True  True]
37301 0.18616584246137866 -3.0 [False False  True  True  True False]
37401 0.18192819052385492 -3.0 [Fa

48401 0.01445106984253965 -3.0 [ True False  True False  True False]
48501 0.01412212333276177 -3.0 [ True False  True False  True False]
48601 0.013800664559703323 -3.0 [ True False  True False  True False]
48701 0.013486523081667822 -3.0 [ True False  True False  True False]
48801 0.013179532336685458 -3.0 [ True False  True False  True False]
48901 0.012879529554199739 -3.0 [ True False  True False  True False]
49001 0.012586355668764383 -3.0 [ True False  True False  True False]
49101 0.01229985523570472 -3.0 [ True False  True False  True False]
49201 0.012019876348698859 -3.0 [ True False  True False  True False]
49301 0.01174627055923495 -3.0 [ True False  True False  True False]
49401 0.011478892797901817 -3.0 [ True False  True False  True False]
49501 0.011217601297471243 -3.0 [ True False  True False  True False]
49601 0.010962257517731092 -3.0 [ True False  True False  True False]
49701 0.010712726072029475 -3.0 [ True False  True False  True False]
49801 0.0104688746554909

In [9]:
N = 1000
t = np.arange(N+1)
T_i = 1000
T_f = 0.001

T = T_i * ((T_f/T_i) ** (t/N))

E_list = []

for t in range(N):
    # take a look at the abstract_udmis.py file to see how mc_step works
    temp = T[t]
    
    E_list.append(udmis.mc_step(T=temp))
    

        
    if t % 100 == 1:
        print(t,temp, E_list[t], udmis.occupations)

        if (E_list[t] == E_list[t-1])&(E_list[t-1] == E_list[t-2])&(E_list[t] <= min(E_list)):
            #print(E_list)
            break

1 986.2794856312105 -1.65 [False  True  True  True False False]
101 247.74220576332854 -0.9999999999999996 [False False False False False  True]
201 62.23002851691594 -1.65 [False False  True  True False  True]
301 15.631476426409542 -1.65 [ True False False  True  True False]
401 3.926449353995997 -2.65 [ True False  True False  True  True]
501 0.9862794856312105 -1.2999999999999998 [ True  True False  True  True False]
601 0.2477422057633286 -3.0 [False False  True  True  True False]


In [16]:
N = 1000
t = np.arange(N+1)
T_i = 1000
T_f = 0.001

T = T_i * ((T_f/T_i) ** (t/N))

for t in range(N):
    # take a look at the abstract_udmis.py file to see how mc_step works
    temp = T[t]
    E = udmis.mc_step(T=temp)
    
    if t % 100 == 1:
        print(t,temp, E, udmis.occupations)

1 986.2794856312105 -1.0 [False False False  True False False]
101 247.74220576332854 -2.0 [ True False False False  True False]
201 62.23002851691594 0.7500000000000004 [ True  True  True  True  True  True]
301 15.631476426409542 -2.0 [ True  True False False False False]
401 3.926449353995997 -2.65 [ True False  True  True  True False]
501 0.9862794856312105 -1.65 [ True  True False False  True False]
601 0.2477422057633286 -2.65 [False False  True  True  True  True]
701 0.06223002851691598 -3.0 [False False  True False  True  True]
801 0.01563147642640953 -3.0 [False False  True  True  True False]
901 0.003926449353995997 -3.0 [False False  True  True  True False]
